# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

*Вызываем библиотеку pandas*

In [30]:
import pandas as pd

*Методом read_scv() прочтем датасет, передадим функции credits. Выводим на экран первые двадцать строк таблицы для ознакомления*

In [31]:
credits = pd.read_csv('/datasets/data.csv')
credits.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


*Первое, что бросается в глаза - разный регистр в столбце education и family_status, значения в days_employed, а также NaN в total_income*

## Шаг 2. Предобработка данных

### Обработка пропусков

*Методом info() просмотрим информацию о таблице*

In [32]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


*В столбцах days_employed и total_income имеются нулевые значения. Удалять их не будем, так как это испортит выборку, поэтому заменим их на медианное значение.Также сначала преобразуем значения в столбце days_employed в абсоютные методом abs()*

In [33]:
credits['days_employed'] = credits['days_employed'].abs()
credits

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


*Находим медианы для каждого столбца*

In [34]:
credits['days_employed'].median()

2194.220566878695

In [35]:
credits['total_income'].median()

145017.93753253992

*Заменяем методом fillna(). В данных столбцах невероятные значения, а в days_employed отрицательные. Мы не будем уделять этому внимание, вероятнее всего, так как это повсеместно, причиной тому служит неверный способ ввода данных. Необходимо сообщить об этом коллегам.*

In [36]:
credits['days_employed'] = credits['days_employed'].fillna(credits['days_employed'].median())
credits['total_income'] = credits['total_income'].fillna(credits['total_income'].median())

*Методом info() просмотрим информацию о таблице. Проверяем наше предыдущее действие.*

In [37]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB



### Замена типа данных

*Мы заметили, что типы данных в используемых столбцах имеют вещественный тип float.Заменим их на целочисленный int методом astype()*


In [38]:
credits['days_employed'] = credits['days_employed'].astype('int')
credits['total_income'] = credits['total_income'].astype('int')

*Проверим предыдущее действие*

In [40]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


<div class="alert alert-danger">
    
<b> Комментарий от ревьюера</b>
    
~Перед тем, как переходить к обработке дубликатов, давай подробно рассмотрим значения наших признаков и поправим их при необходимости. Например, в признаке children есть отрицательные значения, хотя такого быть не должно~
</div>

### Обработка дубликатов

*В нашей таблице credits имеются дубликаты, но прежде чем работать с ними необходимо изменить значение, а именно, привести к нижнему регистру. Данная проблема хорошо видна в столбце education и family_status.Для работы используем метод str.lower()*

In [41]:
credits['education'] = credits['education'].str.lower()
credits['family_status'] = credits['family_status'].str.lower()

In [42]:
credits['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [43]:
credits.query('children == -1').groupby('family_status').count().reset_index()

,family_status,children,days_employed,dob_years,education,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,в разводе,4,4,4,4,4,4,4,4,4,4,4
1,вдовец / вдова,4,4,4,4,4,4,4,4,4,4,4
2,гражданский брак,5,5,5,5,5,5,5,5,5,5,5
3,женат / замужем,29,29,29,29,29,29,29,29,29,29,29
4,не женат / не замужем,5,5,5,5,5,5,5,5,5,5,5


*Также избавимся от отрицательных значений в столбце children*

In [44]:
credits['children'] = credits['children'].abs()

*Найдем дубликаты и их количество в нашей таблице методом duplicated().sum()*

In [45]:
credits.duplicated().sum()

71

*Удаляем дубликаты методом drop_duplicated(). Аргумент drop используем со значением True, чтобы не создавать столбцы со старыми значениями индексов.*

In [46]:
credits = credits.drop_duplicates().reset_index(drop = True)

*Проверим последнее действие.*

In [47]:
credits.duplicated().sum()

0

*Причин появления дубликатов может быть несколько и все они зависят от статуса клинта банка, а именно категоральные переменные(образование, вида дохода, пола, семейного положения).*

### Лемматизация

*Для улучшения анализа выведем леммы в столбце с целями получения кредита. Для этого используем библиотеку pymystem3, но сначала преобразуем данные столбца purpose в текст с помощью ' ' методом join*

In [48]:
text = ' '.join(credits['purpose'].tolist())

*Импортируем библиотеку и лемматизируем текст*

In [49]:
from pymystem3 import Mystem
m = Mystem()
lemmas = m.lemmatize(text)

*Для подсчёта количества значений в списке используем  контейнер Counter из модуля collections.*

In [50]:
from collections import Counter
print(Counter(lemmas))

Counter({' ': 55023, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


*В таблицу credits добавляем новый признак, в который записываем леммы признака purpose. Потом создадим функцию lems(purpose) для уточнения вида кредита по всем леммам*



In [51]:
credits['lemmatized'] = credits['purpose'].apply(m.lemmatize)
credits

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatized
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,"[операция, , с, , жилье, \n]"
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]"
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,"[недвижимость, \n]"
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]"


*В таблицу credits добавляем новый столбец purpose_type, в который записываем виды кредита, полученные функцией lems(purpose)*


In [52]:
def lems(purpose):
    for row in purpose:
        if 'недвижимость' in purpose or 'жилье' in purpose or 'жилой' in purpose:
            purpose_type = 'недвижимость'
        if 'автомобиль' in purpose:
            purpose_type = 'автомобиль'
        if 'образование' in purpose:
            purpose_type = 'образование'
        if 'свадьба' in purpose:
            purpose_type = 'свадьба'
        return purpose_type
credits['purpose_type'] = credits['lemmatized'].apply(lems)
credits

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatized,purpose_type
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]",автомобиль
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,"[недвижимость, \n]",недвижимость
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]",автомобиль


### Категоризация данных

*Напишем функцию children_group(children), оценивающая виды семей по количеству детей в семье: бездетные, однодетная, малодетная, многодетная*

In [53]:
def children_group(children):
    if children == 0:
        return 'бездетная'
    if children == 1:
        return 'однодетная'
    if children == 2:
        return 'малодетная'
    return 'многодетная'

*Добавим к таблице credits столбец children_group, где хранятся результаты нашей функции*

In [54]:
credits['children_group'] = credits['children'].apply(children_group)
credits

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatized,purpose_type,children_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,однодетная
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,однодетная
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,бездетная
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,многодетная
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,бездетная
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость,однодетная
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]",автомобиль,бездетная
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,"[недвижимость, \n]",недвижимость,однодетная
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]",автомобиль,многодетная


*С помощью метода pivot_table создадим таблицу, где вычислим доли показателя debt, группируя по видам семей с детьми*


In [55]:
credits1 = credits.pivot_table(index='children_group', values='debt', aggfunc='mean')
credits1

,debt
children_group,
бездетная,0.075438
малодетная,0.094542
многодетная,0.085526
однодетная,0.091658


*По результатам таблицы мы можем увидеть, что в семьях без детей меньше всего кредитная задолженность. На чем делаем вывод, что с ростом количества детей вероятность задолженности растет*


*С помощью метода pivot_table создадим таблицу, где вычислим доли показателя debt, группируя по семейному положению*

In [56]:
credits2 = credits.pivot_table(index='family_status', values='debt', aggfunc='mean')
credits2

,debt
family_status,
в разводе,0.071130
вдовец / вдова,0.065693
гражданский брак,0.093471
женат / замужем,0.075452
не женат / не замужем,0.097509


*По результатам данной таблицы мы можем сделать вывод, что задолженность по кредиту высока у клиентов холостых или не зарегестрировавших свои отношения*


*Напишем функцию totalinc_group(total_income), оценивающая виды семей по уровню дохода: нижний, средний, высокий, сверхвысокий*

In [57]:
def totalinc_group(total_income):
    if total_income <= 40000:
        return 'нижний'
    if total_income <= 100000:
        return 'средний'
    if total_income <= 1000000:
        return 'высокий'
    return 'сверхвысокий'

*Добавим к таблице credits столбец totalinc_group, где хранятся результаты нашей функции*

In [58]:
credits['totalinc_group'] = credits['total_income'].apply(totalinc_group)
credits

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatized,purpose_type,children_group,totalinc_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,однодетная,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,однодетная,высокий
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,бездетная,высокий
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,многодетная,высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,бездетная,высокий
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость,однодетная,высокий
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]",автомобиль,бездетная,высокий
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,"[недвижимость, \n]",недвижимость,однодетная,средний
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]",автомобиль,многодетная,высокий


*С помощью метода pivot_table создадим таблицу, где вычислим доли показателя debt, группируя по уровню дохода*

In [59]:
credits3 = credits.pivot_table(index='totalinc_group', values='debt', aggfunc='mean')
credits3

,debt
totalinc_group,
высокий,0.081634
нижний,0.080000
сверхвысокий,0.080000
средний,0.079299


*Результат таблицы указывает, что люди с высоким уровнем дохода имеют большую долю кредитного долга. Также можно сделать вывод о высокой задолженности всех 4х групп клиентов.


*С помощью метода pivot_table создадим таблицу, где вычислим доли показателя debt, группируя по цели кредита*


In [60]:
credits4 = credits.pivot_table(index='purpose_type', values='debt', aggfunc='mean')
credits4

,debt
purpose_type,
автомобиль,0.093590
недвижимость,0.072334
образование,0.092200
свадьба,0.080034


В данной таблице выделяется две цели(автомобиль и образование), которые имеют высокую, почти одинаковую, кредитную задолженность, относительно двух других.


## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

**Небольшая зависимость все же присутсвует, но разница не велика(0.06 - 0.03)**

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

**Есть отрицательная зависимость в сторону клиентов, живущих в гражданском браке или не женат/ не замужем**

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

**В целом, нет существенной зависимости между клиентами с разным уровнем дохода**

- Как разные цели кредита влияют на его возврат в срок?

**Выявлены 4 цели получения кредита. При целях автомобиль и образование имееться высокий долг**

## Шаг 4. Общий вывод

**В результате работы над проектом мы выяснили, что бездетные семьи имеют меньшую кредитную нагрузку, женатые чаще холостых возвращают кредиты в срок, не важно какой уровень дохода у клиентов - это не зависит на возврат кредита, а кредиты на свадьбу или операции с недвижимостью будут выплачены быстрее остальных**